# Kernel K-means and Spectral Clustering

In [22]:
# Deal with data
import numpy as np
import cv2

img1 = cv2.imread('image1.png')
img2 = cv2.imread('image2.png')

# Flatten the image; img[x] as an array with shape (100, 3)
def process(img):
    X, Y, C = img.shape
    flatten = np.zeros((X * Y, C))
    for x in range(X):
        flatten[x*Y:(x+1)*Y] = img[x]
    return flatten,X,Y

In [23]:
# Kernel functions
from scipy.spatial.distance import pdist,squareform

# Get kernel function: k(x,x')= exp(-r_s*||S(x)-S(x')||**2)* exp(-r_c*||C(x)-C(x')||**2)
def kernel(X,r_s=1,r_c=1):
    '''
    X: (10000,rgb=3) ndarray
    r_s: gamma of spacial
    r_c: gamma of color
    return : K (10000,10000) array
    '''
    n=len(X)
    S=np.zeros((n,2)) # S is the spacial information: [0,1],[0,2]...[99,99]
    for i in range(n):
        S[i]=[i//100,i%100]
    K=squareform(np.exp(-r_s*pdist(S,'sqeuclidean')))*squareform(np.exp(-r_c*pdist(X,'sqeuclidean')))
    return K

In [24]:
# Visualization
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from array2gif import write_gif

# predefine colormap
colormap = np.random.choice(range(256),size=(100,3))

def visual(C,k,H,W):
    '''
    C: (10000) array, classes of each datapoints
    k: num of clusters
    H: image height
    W: image width
    return : (H,W,3) nrray
    '''
    colors= colormap[:k,:]
    ans=np.zeros((H,W,3))
    for h in range(H):
        for w in range(W):
            ans[h,w,:]=colors[C[h*W+w]]

    return ans.astype(np.uint8)

# For 3-d datas
def show_eigenvector(x,y,z,C):
    '''
    x: (#datapoint) array
    y: (#datapoint) array
    z: (#datapoint) array
    C: (#datapoint) array, class
    '''
    fig=plt.figure(figsize =(15,15))
    ax=fig.add_subplot(111,projection='3d')
    markers=['+','o','^']
    for marker,i in zip(markers,np.arange(3)):
        ax.scatter(x[C==i],y[C==i],z[C==i],marker=marker)

    ax.set_xlabel('eigenvector 1-d')
    ax.set_ylabel('eigenvector 2-d')
    ax.set_zlabel('eigenvector 3-d')
    plt.show()

def gif(dataset,gif_name):
    for i in range(len(dataset)):
        dataset[i] = dataset[i].transpose(1, 0, 2)
    write_gif(dataset, gif_name, fps=2)

Use diffrent initialized method :'Forgy Initialization','Random Partition Initialization','kmeans++ Initialization'

Reference: 
    
https://medium.com/analytics-vidhya/comparison-of-initialization-strategies-for-k-means-d5ddd8b0350e

In [25]:
# K-means function

EPS=1e-9
# inital type: 'forgy','random','k_means_plusplus'

def initial_cluster(X,k,init):
    '''
    X: data, array
    k: num of clusters
    return: Cluster: (k,#features) array
    Kij: cluster i's j-dim value
    '''
    cluster = np.zeros((k, X.shape[1]))
    if init == 'forgy':
        cluster=X[np.random.choice(range(X.shape[0]), replace = False, size = k), :]

    elif init =='random':
        mu=np.mean(X,axis=0)
        std=np.std(X,axis=0)
        for i in range(X.shape[1]):
            cluster[:,i]=np.random.normal(mu[i],std[i],size=k)

    else: # init =='k_means_plusplus'
        # pick first cluster_mean
        cluster[0]=X[np.random.randint(low=0,high=X.shape[0],size=1),:]
        # pick k-1 cluster_mean
        for c in range(1,k):
            dist=np.zeros((len(X),c))
            for i in range(len(X)):
                for j in range(c):
                    dist[i,j]=np.sqrt(np.sum((X[i]-cluster[j])**2))
            dist_min=np.min(dist,axis=1)
            dist_sum=np.sum(dist_min)*np.random.rand()
            for i in range(len(X)):
                dist_sum-=dist_min[i]
                if dist_sum<=0:
                    cluster[c]=X[i]
                    break

    return cluster


def k_means(X,k,H,W,init):
    '''
    X: (#datapoint,#features) array
    k: num of clusters
    H: image H
    W: image W
    return: (#datapoint) ndarray, Ci: belonging class of each data point
    '''
    mu=initial_cluster(X,k,init)

    # Classes of each Xi
    C=np.zeros(len(X),dtype=np.uint8)
    pic=[]
    diff=1e5
    count=1
    while diff>EPS :
        # E-step
        for i in range(len(X)):
            dist=[]
            for j in range(k):
                dist.append(np.sqrt(np.sum((X[i]-mu[j])**2)))
            C[i]=np.argmin(dist)

        # M-step
        New_mu=np.zeros(mu.shape)
        for i in range(k):
            belong=np.argwhere(C==i).reshape(-1)
            for j in belong:
                New_mu[i]=New_mu[i]+X[j]
            if len(belong)>0:
                New_mu[i]=New_mu[i]/len(belong)

        diff = np.sum((New_mu - mu)**2)
        mu=New_mu

        # visualize
        p = visual(C,k,H,W)
        pic.append(p)
        print('iteration {}'.format(count))
        for i in range(k):
            print('k={}: {}'.format(i + 1, np.count_nonzero(C == i)))
        print('diff {}'.format(diff))
        print('-------------------')
        cv2.imshow('', p)
        cv2.waitKey(1)

        count+=1
    return C,pic


# Kernel K-means 

In [46]:
# Kernel k-means main

image_flat,H,W=process(img1)
gamma_s=0.001
gamma_c=0.001
k=3  # k clusters
k_means_init='k_means_plusplus'

Gram_matrix=kernel(image_flat,gamma_s,gamma_c)
belongings,pic=k_means(Gram_matrix,k,H,W,init=k_means_init)
gif(pic,'img1_k3_k++.gif')


cv2.waitKey(0)
cv2.destroyAllWindows()

iteration 1
k=1: 7917
k=2: 874
k=3: 1209
diff 176.18626326557026
-------------------
iteration 2
k=1: 7270
k=2: 1242
k=3: 1488
diff 15.226061594678095
-------------------
iteration 3
k=1: 7162
k=2: 1386
k=3: 1452
diff 3.555003050070598
-------------------
iteration 4
k=1: 7141
k=2: 1448
k=3: 1411
diff 1.612664853198617
-------------------
iteration 5
k=1: 7144
k=2: 1442
k=3: 1414
diff 0.7109966503673238
-------------------
iteration 6
k=1: 7166
k=2: 1415
k=3: 1419
diff 0.34093178629236454
-------------------
iteration 7
k=1: 7181
k=2: 1389
k=3: 1430
diff 0.18678211826889224
-------------------
iteration 8
k=1: 7187
k=2: 1359
k=3: 1454
diff 0.16159428243336108
-------------------
iteration 9
k=1: 7193
k=2: 1326
k=3: 1481
diff 0.15061763395927258
-------------------
iteration 10
k=1: 7196
k=2: 1310
k=3: 1494
diff 0.08962621039463431
-------------------
iteration 11
k=1: 7195
k=2: 1307
k=3: 1498
diff 0.024877371017589994
-------------------
iteration 12
k=1: 7195
k=2: 1304
k=3: 1501
diff 

#  Spectral Clustering

normalized cut

In [63]:
# main of normalized cut

EPS=1e-9
# inital type: 'forgy','random','k_means_plusplus'

image_flat,HEIGHT,WIDTH=process(img2)
gamma_s=0.001
gamma_c=0.001
k_means_init='random'
k=4  # k clusters

# similarity matrix
W=kernel(image_flat,gamma_s,gamma_c)
# degree matrix
D=np.diag(np.sum(W,axis=1))
# approximation of normalized cut
L=D-W
D_inverse_square_root=np.diag(1/np.diag(np.sqrt(D)))
L_sym=D_inverse_square_root@L@D_inverse_square_root

eigenvalue,eigenvector=np.linalg.eig(L_sym)
sort_index=np.argsort(eigenvalue)
# U
U=eigenvector[:,sort_index[1:1+k]]

# k-means
belonging,pic=k_means(U,k,HEIGHT,WIDTH,init=k_means_init)

gif(pic,'img2_s4_k++_random.gif')
if k==3:
    show_eigenvector(U[:,0],U[:,1],U[:,2],belonging)

    
    
cv2.waitKey(0)
cv2.destroyAllWindows()


iteration 1
k=1: 1851
k=2: 2751
k=3: 2843
k=4: 2555
diff 0.00047572818525615907
-------------------
iteration 2
k=1: 2025
k=2: 2039
k=3: 3359
k=4: 2577
diff 4.333222338587826e-05
-------------------
iteration 3
k=1: 2122
k=2: 2003
k=3: 3245
k=4: 2630
diff 3.382746510089444e-05
-------------------
iteration 4
k=1: 2651
k=2: 2073
k=3: 2444
k=4: 2832
diff 7.46614706783071e-05
-------------------
iteration 5
k=1: 2843
k=2: 2219
k=3: 2078
k=4: 2860
diff 2.44687874892607e-05
-------------------
iteration 6
k=1: 2806
k=2: 2328
k=3: 2024
k=4: 2842
diff 2.4935415494848033e-06
-------------------
iteration 7
k=1: 2797
k=2: 2370
k=3: 2011
k=4: 2822
diff 5.469928553863925e-07
-------------------
iteration 8
k=1: 2794
k=2: 2390
k=3: 2010
k=4: 2806
diff 1.132820125867934e-07
-------------------
iteration 9
k=1: 2791
k=2: 2405
k=3: 2010
k=4: 2794
diff 5.367940211219812e-08
-------------------
iteration 10
k=1: 2793
k=2: 2414
k=3: 2010
k=4: 2783
diff 2.252744408106521e-08
-------------------
iteration

ratio cut (unnormalize Laplacian)

In [67]:
# main of ratio cut 
# inital type: 'forgy','random','k_means_plusplus'

image_flat,HEIGHT,WIDTH=process(img2)
gamma_s=0.001
gamma_c=0.001
k_means_init='forgy'
k=4  # k clusters

# similarity matrix
W=kernel(image_flat,gamma_s,gamma_c)
# degree matrix
D=np.diag(np.sum(W,axis=1))
# approximation fo ratio cut
L=D-W

eigenvalue,eigenvector=np.linalg.eig(L)
sort_index=np.argsort(eigenvalue)
# U
U=eigenvector[:,sort_index[1:1+k]]

# k-means
belonging,pic=k_means(U,k,HEIGHT,WIDTH,init=k_means_init)

gif(pic,'img2_s4_forgy_un.gif')
if k==3:
    show_eigenvector(U[:,0],U[:,1],U[:,2],belonging)

    
    
cv2.waitKey(0)
cv2.destroyAllWindows()

iteration 1
k=1: 3526
k=2: 2308
k=3: 3562
k=4: 604
diff 0.0002648073233005046
-------------------
iteration 2
k=1: 2714
k=2: 2946
k=3: 3261
k=4: 1079
diff 2.1016996999729784e-05
-------------------
iteration 3
k=1: 2387
k=2: 3219
k=3: 3129
k=4: 1265
diff 4.228978789386528e-06
-------------------
iteration 4
k=1: 2261
k=2: 3318
k=3: 3012
k=4: 1409
diff 9.397002946992541e-07
-------------------
iteration 5
k=1: 2215
k=2: 3375
k=3: 2846
k=4: 1564
diff 9.57039520369343e-07
-------------------
iteration 6
k=1: 2197
k=2: 3467
k=3: 2420
k=4: 1916
diff 5.27056027295302e-06
-------------------
iteration 7
k=1: 2197
k=2: 3622
k=3: 2202
k=4: 1979
diff 2.4947015218312103e-06
-------------------
iteration 8
k=1: 2208
k=2: 3749
k=3: 2149
k=4: 1894
diff 7.716818533051887e-07
-------------------
iteration 9
k=1: 2216
k=2: 3820
k=3: 2126
k=4: 1838
diff 2.4398731946981494e-07
-------------------
iteration 10
k=1: 2223
k=2: 3855
k=3: 2114
k=4: 1808
diff 6.769483657067893e-08
-------------------
iteration